# MTH 448 Project 4

## Race with Bayes

## Introduction

For project four, we need to analyze a data set. This dataset contains 26,000 marathon results. In this project, we will use Bayes' theorem to predict gender based on marathon finish time. Usually, a feature may be a lack of objectivity in our prediction results and vary in accuracy. Therefore, in this project, we can also add other features such as age factor. The accuracy of the final prediction result can be observed through the factors of different variables. Also in this project we will use KNN (K- Nearest Neighbor) to compare the accuracy of the two algorithms. When we start to exlpore the KDE and prediction model, we have to understand what is Bayes' Theorem.


# Methodology

### Bayes' Thorem with KDE:

Before we start applying Bayes' theorem, we can understand also what is Bayes' theorem. Bayes' Theorem is based on the 18th century English mathematician, Thomas Bayes. The theorem is derived from a mathematical formula used in probability theory for missing conditional probabilities. Conditional probability is the likelihood of predicting that this outcome may occur in a particular situation based on a previous mediator. So in Project4, we will use Bayes' theorem to predict gender from marathon times. This is Bayes' Theorem formula:



$$
P(A \mid B) = \frac{P(B \mid A) \cdot P(A)}{P(B)}
$$


* $P$(A|B) is the  is the posterior probability: the probability of  event A occurring given that B is true.
* $P$(B|A) is  is the likelihood: the probability of event B occurring given that A is true.
* $P$(A) is the prior probability of A.
* $P$(B) is the marginal probability of B.

Hence, for our project, the Bayes Theorem will be:
$$
P(female \mid time) = \frac{P(time \mid female) \cdot P(female)}{P(time)}
$$


First of all, we have to import the Libraries, so that we can get through the csv file.

In [264]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We should load the .csv file to see the data.

In [265]:
data = pd.read_csv('marathon_results.csv')  

In [266]:
data

,Age,M/F,Country,5K,10K,15K,20K,Half,25K,30K,35K,40K,Finish,Pace,Overall,Gender,Division
0,25,M,ETH,00:14:43,00:29:43,00:44:57,01:00:29,01:04:02,01:16:07,01:32:00,01:47:59,02:02:39,02:09:17,00:04:56,1,1,1
1,30,M,ETH,00:14:43,00:29:43,00:44:58,01:00:28,01:04:01,01:16:07,01:31:59,01:47:59,02:02:42,02:09:48,00:04:58,2,2,2
2,29,M,KEN,00:14:43,00:29:43,00:44:57,01:00:29,01:04:02,01:16:07,01:32:00,01:47:59,02:03:01,02:10:22,00:04:59,3,3,3
3,28,M,KEN,00:14:43,00:29:44,00:45:01,01:00:29,01:04:02,01:16:07,01:32:00,01:48:03,02:03:47,02:10:47,00:05:00,4,4,4
4,32,M,KEN,00:14:43,00:29:44,00:44:58,01:00:28,01:04:01,01:16:07,01:32:00,01:47:59,02:03:27,02:10:49,00:05:00,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,64,F,USA,00:50:15,01:43:31,02:36:53,03:32:26,03:43:46,04:25:53,05:19:44,06:17:19,07:13:34,07:38:56,00:17:31,26594,12015,269
26294,61,F,USA,00:48:36,01:39:39,02:39:13,03:35:58,03:47:55,04:32:44,05:31:58,06:28:56,07:26:19,07:51:30,00:17:59,26595,12016,270
26295,66,F,USA,00:53:03,01:47:16,02:41:45,03:37:07,03:48:21,04:33:51,05:38:56,06:38:51,07:36:18,07:59:33,00:18:18,26596,12017,91
26296,53,M,USA,00:49:04,01:40:12,02:33:31,03:31:41,03:43:35,04:29:20,05:31:11,06:33:35,07:35:38,08:00:37,00:18:20,26597,14580,2055


Through this chart, it indicate too much columns. However, we only need some parts of them for the part one.
Firstly, we can try the data processing, convert to "gender" to binary. '0' mean male and '1' means female.

In [267]:
data = data.dropna()

In [268]:
data['M/F'] = data['M/F'].map({'M': 0, 'F': 1}) # binary change M means male, F means femal

Because according to our “traing_set" and "testing_set" we can see that our database is very large. And if we want to calculate the 1-Dimension KDE we need to know the end time of the marathon race. But by displaying the data we can see that the finish time is recorded in seconds and minutes. So, for the sake of our calculation. We can generate a separate column named “Time(s)” and calculate it in seconds. This will help us to measure and compare the time better.

In [269]:
def FinishToTime(F):
    Flist = F.split(':')
    secs = int(Flist[2]) + 60*int(Flist[1]) +60*60 *int(Flist[0])
    return secs

In [270]:
data['Time(s)'] = data['Finish'].map(FinishToTime)

In [271]:
data

,Age,M/F,Country,5K,10K,15K,20K,Half,25K,30K,35K,40K,Finish,Pace,Overall,Gender,Division,Time(s)
0,25,0,ETH,00:14:43,00:29:43,00:44:57,01:00:29,01:04:02,01:16:07,01:32:00,01:47:59,02:02:39,02:09:17,00:04:56,1,1,1,7757
1,30,0,ETH,00:14:43,00:29:43,00:44:58,01:00:28,01:04:01,01:16:07,01:31:59,01:47:59,02:02:42,02:09:48,00:04:58,2,2,2,7788
2,29,0,KEN,00:14:43,00:29:43,00:44:57,01:00:29,01:04:02,01:16:07,01:32:00,01:47:59,02:03:01,02:10:22,00:04:59,3,3,3,7822
3,28,0,KEN,00:14:43,00:29:44,00:45:01,01:00:29,01:04:02,01:16:07,01:32:00,01:48:03,02:03:47,02:10:47,00:05:00,4,4,4,7847
4,32,0,KEN,00:14:43,00:29:44,00:44:58,01:00:28,01:04:01,01:16:07,01:32:00,01:47:59,02:03:27,02:10:49,00:05:00,5,5,5,7849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,64,1,USA,00:50:15,01:43:31,02:36:53,03:32:26,03:43:46,04:25:53,05:19:44,06:17:19,07:13:34,07:38:56,00:17:31,26594,12015,269,27536
26294,61,1,USA,00:48:36,01:39:39,02:39:13,03:35:58,03:47:55,04:32:44,05:31:58,06:28:56,07:26:19,07:51:30,00:17:59,26595,12016,270,28290
26295,66,1,USA,00:53:03,01:47:16,02:41:45,03:37:07,03:48:21,04:33:51,05:38:56,06:38:51,07:36:18,07:59:33,00:18:18,26596,12017,91,28773
26296,53,0,USA,00:49:04,01:40:12,02:33:31,03:31:41,03:43:35,04:29:20,05:31:11,06:33:35,07:35:38,08:00:37,00:18:20,26597,14580,2055,28837


Follow the Bayes Theorem:
$$
P(female \mid time) = \frac{P(time \mid female) \cdot P(female)}{P(time)}
$$
 We will get $P$(time| female) and $P$(time |male) using KDE.
 $P$(female) =. proportion of females in training dataset.

# 1. Split the Dataset and Compute KDE

Then We need to split the dataset into two different sets, training set and testing set.
Before we are going to split the data set, we have to understand why we have to split the dataset.
* Training set: The training set is used to train the model and estimate the conditional distribution for KDE P(time,age | gender). Hence, we need to train the model,for the Bayesian classifier, we also need to compute the 
   likelihood and posterior based on the training data. Moreover, because we wanted to predict males and females based on marathon finish time.Thus,we need to calculate prior proabilities P(female) and P(male) by using training model.Finally, we can build a K-NN model by storeing vectors(time and age).
   
* Testing Set: Testing set plays an very important role when we want to know the mode is overfitting or underfitting. We also can oberseve the different between Bayesian theorm and K-NN modelBecause according to our “traing_set" and "testing_set" we can see that our database is very large. And if we want to calculate the 1-Dimension KDE we need to know the end time of the marathon race. But by displaying the data we can see that the finish time is recorded in seconds and minutes. So, for the sake of our calculation. We can generate a separate column named “Time(s)” and calculate it in seconds. This will help us to measure and compare the time better.

In [272]:
# Import the "train_test_split" library from scikit-learn model.
from scipy.stats import gaussian_kde
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Then split the date into training set and testing set.

In [273]:
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

In [274]:
train_set

,Age,M/F,Country,5K,10K,15K,20K,Half,25K,30K,35K,40K,Finish,Pace,Overall,Gender,Division,Time(s)
25456,28,1,USA,00:33:25,01:06:55,01:40:48,02:16:47,02:24:40,02:56:06,03:37:36,04:22:01,05:04:14,05:21:56,00:12:17,25720,11516,5726,19316
14983,36,1,USA,00:26:03,00:51:36,01:17:06,01:43:05,01:48:39,02:09:13,02:36:39,03:05:01,03:33:18,03:45:25,00:08:36,15062,5252,3468,13525
18009,46,0,USA,00:25:06,00:49:17,01:13:55,01:40:12,01:45:48,02:05:27,02:33:39,03:05:04,03:44:09,03:56:36,00:09:02,18135,10978,2045,14196
22326,57,1,CAN,00:28:27,00:56:46,01:25:39,01:56:01,02:02:31,02:27:08,03:00:08,03:35:33,04:09:32,04:25:30,00:10:08,22522,9735,449,15930
10114,59,0,USA,00:24:55,00:49:23,01:13:49,01:38:24,01:43:44,02:02:57,02:28:14,02:54:10,03:19:09,03:30:00,00:08:01,10147,7738,338,12600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,45,1,USA,00:29:04,00:57:44,01:26:39,01:59:09,02:05:33,02:28:48,03:01:45,03:34:01,04:06:02,04:19:11,00:09:54,21760,9312,1484,15551
5390,41,0,GBR,00:21:30,00:42:50,01:04:37,01:26:41,01:31:34,01:49:31,02:13:16,02:37:42,03:02:37,03:13:23,00:07:23,5403,4826,929,11603
860,42,0,USA,00:20:16,00:39:56,01:00:02,01:20:08,01:24:32,01:40:10,02:00:35,02:20:57,02:40:53,02:49:22,00:06:28,862,830,74,10162
15795,40,0,USA,00:23:11,00:45:46,01:08:46,01:31:56,01:36:56,01:55:58,02:22:00,02:50:29,03:26:32,03:48:26,00:08:43,15889,10103,1612,13706


In [275]:
test_set

,Age,M/F,Country,5K,10K,15K,20K,Half,25K,30K,35K,40K,Finish,Pace,Overall,Gender,Division,Time(s)
15429,23,1,USA,00:26:49,00:51:51,01:17:36,01:43:39,01:49:18,02:10:09,02:37:47,03:06:07,03:34:44,03:47:06,00:08:40,15515,5542,3593,13626
14903,44,0,USA,00:23:34,00:47:08,01:10:42,01:35:27,01:40:53,02:01:03,02:28:22,02:59:41,03:31:31,03:45:09,00:08:36,14982,9780,1591,13509
2973,42,0,USA,00:21:31,00:42:29,01:03:28,01:24:28,01:28:57,01:45:13,02:07:13,02:29:28,02:51:24,03:01:43,00:06:56,2980,2807,433,10903
15267,34,1,USA,00:25:46,00:52:50,01:19:37,01:46:32,01:52:42,02:14:05,02:42:31,03:09:20,03:35:19,03:46:29,00:08:39,15347,5436,3546,13589
22153,51,1,USA,00:29:53,00:59:05,01:29:02,01:59:19,02:05:58,02:30:16,03:03:14,03:37:03,04:09:48,04:24:00,00:10:05,22347,9633,976,15840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245,22,0,USA,00:24:09,00:45:42,01:07:28,01:29:24,01:34:14,01:52:00,02:15:14,02:38:41,03:02:56,03:12:52,00:07:22,5258,4717,2908,11572
11242,37,1,USA,00:24:15,00:48:46,01:13:27,01:38:10,01:43:34,02:03:04,02:28:38,02:55:10,03:21:25,03:33:32,00:08:09,11284,3032,2333,12812
2814,57,0,USA,00:21:26,00:42:19,01:03:15,01:24:13,01:28:50,01:45:16,02:06:50,02:28:58,02:51:09,03:00:51,00:06:54,2820,2660,33,10851
12527,50,0,USA,00:22:43,00:45:32,01:09:21,01:33:04,01:38:12,01:57:02,02:23:46,02:52:37,03:23:51,03:37:28,00:08:18,12582,8797,1184,13048


In [276]:
print("Data preparation complete.")
print(f"Training samples: {len(train_set)}, Test samples: {len(test_set)}")

Data preparation complete.
Training samples: 21038, Test samples: 5260


After we split into two different data sets, we can extraxt the time from two genders: male and female.
    

# 2. Know about Guassian KDE

Before we apply KDE to estimate for 1Dimension KDE,we have to try to understand why we should use KDE here.
KDE(Gaussian Kernel Density Estimation), Gaussian KDE is a non-parametric method to estimate the probaility density 
function(PDF) of a random varibale.

* What is "Non parametric"? It means we do not have to assume a specific distribution, for example, normal distribution, exponential distribution.
* Why we use KDE? We want to estimate $P$(time|gender) and help Bayes' Theorem to make probabilitic predictions for the gender.

### 2.1 Compute 1-D KDE Bayesian

We using 1 dimension KDE on finish time to estimate $P$(time|female).
Then we will use Bayes' Theorem to compute accuracy. After that, we can predict the gender for marathon and compute the accuracy. So we can follow the Bayes' Theorem to write the code.
$$
P(female \mid time) = \frac{P(time \mid female) \cdot P(female)}{P(time)}
$$



In [277]:
def MakeKDEs(bandwidth=None,train_set=train_set):
    female_times = train_set[train_set['M/F'] == 1]['Time(s)'].values
    all_times = train_set['Time(s)'].values
    
    
    # Create KDE models with proper bandwidth handling
    try:
        kde_female = gaussian_kde(female_times, bw_method=bandwidth)
        kde_all = gaussian_kde(all_times, bw_method=bandwidth)
    except Exception as e:
        print(f"KDE creation failed: {e}")
        return [0] * len(test_data)
    return kde_female,kde_all

In [278]:
kde_f,kde_all = MakeKDEs()

Then, create the function named "bayes_1d_classifier " with the one feature "Time(s)".

In [279]:
# Function for 1 dimension
def bayes_1d_classifier(time,kde_female=kde_f,kde_all=kde_all):
    # Separate female and male 
    female_times = train_set[train_set['M/F'] == 1]['Time(s)'].values
    all_times = train_set['Time(s)'].values
    
    # Then we can follow the equation to calcuate 
    p_female = len(female_times) / len(all_times)
    p_time = kde_all(time)
    PtimeFem = kde_female(time)
    p_F_result = (PtimeFem*p_female)/p_time
    # We will check p_F_result proability, The "True" value for female, and "False" value for male.
    return p_F_result>0.5
    
    

Use one case to show which gender will coming out. For the value, if the Proabiliy higher than 50%, it will be female and return value should be "True".
On the other hand, the proability less than 50%, it will return value "False", means male.

In [280]:
bayes_1d_classifier([27000])  # Try to use one test case

array([ True])

In [281]:
def TestClassifier(test_set=test_set,kde_female=kde_f,kde_all=kde_all):
    test_true = test_set['M/F'].values
    predMF = bayes_1d_classifier(test_set['Time(s)'],kde_female=kde_female,kde_all=kde_all)
    numcorrect = np.sum(predMF==test_true)
    accuracy = numcorrect/len(test_true)
    return accuracy

In [282]:
TestClassifier()

0.6534220532319391

After we got the result from "TestClassifier", we can estimate the accuracy about "0.65". Then, we need create one more function named "evaluate_1d_classifier" with two
paramters to evaluaye 1-dimensional classifier with different bandwidths.

In [283]:
def evaluate_1d_classifier(train_set, test_set):
    
    bandwidths = [0.1, 0.5, 1.0, 'scott', 'silverman']
    results = []
    
    for bw in bandwidths:
        try:
            preds = bayes_1d_classifier(train_set, test_set, bw)
            acc = accuracy_score(test_set['M/F'], preds)
            results.append((bw, acc))
            print(f"Bandwidth {str(bw):<9}: Accuracy = {acc:.4f}")
        except Exception as e:
            print(f"Bandwidth {str(bw):<9}: Failed - {str(e)}")
            results.append((bw, 0))
    
    # Return best bandwidth
    best_bw, best_acc = max(results, key=lambda x: x[1])
    print(f"\nBest bandwidth: {best_bw} (Accuracy: {best_acc:.4f})")
    return best_bw

Applying Bayesian classification using 1D Gaussian Kernel Density Estimation(KDE) with different bandwidths.
Hence, we need loop over a list of bandwidths to test how KDE changes dependone on the distributions. We set the range at 5 and went to see how it varies among these 5 accuracy values and how big the gap is between them.

In [284]:
bandwidths = [0.1, 0.5, 1.0, 'scott', 'silverman']
accuracies = []
x = np.linspace(7000,30000,1000)
for bw in bandwidths:
    kde_female,kde_all = MakeKDEs(bandwidth=bw)
    #plt.plot(x,kde_female(x),'r-')
    #plt.plot(x,kde_all(x),'b-')
    #plt.show()
    print(bw)
    acc= TestClassifier(kde_female=kde_female,kde_all=kde_all)
    print(acc)
    accuracies.append(acc)

0.1
0.652851711026616
0.5
0.6526615969581749
1.0
0.6361216730038023
scott
0.6534220532319391
silverman
0.6532319391634981


In [285]:
accuracies

[0.652851711026616,
 0.6526615969581749,
 0.6361216730038023,
 0.6534220532319391,
 0.6532319391634981]

Based on our selection of five data sets we can see that the accuracy of 1dimensionKDE is in the range of 63% -65%.

## 2.2 Compute 2-Dimensional KDE

After we did the 1D Guassian KDE, we can use both time and age as input features to performing the 2D
Guassian kernel Density Estimation.(KDE). Why we use 2D KDE here?
* The 1D model only used for time, which may not can catch the full difference between genders.
* Secondly, this 2- Dimension include gae, which can provide more information and improve the accuracy.



The Bayes' theorem:
$$
P(female \mid time, Age) = \frac{P(time, Age \mid female) \cdot P(female)}{P(time, Age)}
$$


Hence, the bayes'theorem also changed follor the two different element,
The KDE is defined over the joint space $P$(finish time, age|gender). 
We need "Marathon finish time" and "Age".
At first, we will follow the same steps from the part 1. Crate a function named 'bayes_2d_classifier' with two parameter.



In [286]:
def Make2DKDEs(bandwidth=None,train_set=train_set):
     
    # Prepare 2D data
    female_data = train_set[train_set['M/F'] == 1][['Time(s)', 'Age']].values.T
    all_data = train_set[['Time(s)', 'Age']].values.T
    
    
    
    # Create KDE models
    kde_female = gaussian_kde(female_data)
    kde_all = gaussian_kde(all_data)
    return kde_female,kde_all

In [287]:
kde_2d_female,kde_2d_all=Make2DKDEs()

Then train KDEs for each class, we already import the library before, hence, we do not have to import this time.
We can create a variable named kde_f2, it means for female 2D KDE and tt estimate $P$($Time$ = t, $Age$ = a|female).
Hence, we also can create a variable named kde_m2 to refer to male.When we use 2D Kernel Density Estimation model to try to predict a Mataton's gender, 
we need to based on the participant's finish time and age. 
Hence, first looping over the testing dataset to make predictions. 

In [288]:
def bayes_2d_classifier(test_set=test_set,kde_2d_female=kde_2d_female,kde_2d_all=kde_2d_all):
    """2D Bayesian classifier using finish times and ages"""
   
    female_data = train_set[train_set['M/F'] == 1][['Time(s)', 'Age']].values.T
    all_data = train_set[['Time(s)', 'Age']].values.T
    # Then we can calculate priors
    p_female = len(female_data) / len(all_data)
    p_time_age = kde_2d_all(test_set)
    p_time_age_F = kde_2d_female(test_set)
    p_F_result = (p_time_age_F*p_female)/p_time_age
    return p_F_result>0.5
    
    
   

We will follow the 2D KDE to check the proability. For the return false, if the value return "True"
that means is female. Otherwise, it will return male.


In [289]:
bayes_2d_classifier([47000, 40]) # This case shows that the 47000sec, and age is 40. It should be a male

/var/folders/jw/4sww0bws1kv_2dyz5c2y6s0h0000gn/T/ipykernel_6596/2475777581.py:10: RuntimeWarning: invalid value encountered in divide
  p_F_result = (p_time_age_F*p_female)/p_time_age


array([False])

In [290]:
bayes_2d_classifier([27536, 64]) # We can check the case from the test_set, it is female.

array([ True])

In [295]:
def TestClassifier2D(test_set=test_set,kde_2d_female=kde_2d_female,kde_2d_all=kde_2d_all):
    test_true = test_set['M/F'].values
    predMF = bayes_2d_classifier(test_set[['Time(s)','Age']],kde_2d_female=kde_2d_female,kde_2d_all=kde_2d_all)
    numcorrect = np.sum(predMF==test_true)
    accuracy = numcorrect/len(test_true)
    return accuracy

In [297]:
TestClassifier2D()

ValueError: points have dimension 5260, dataset has dimension 2

As can be seen by our accuracy data, for 1-dimension kde, the accuracy values are roughly in the 63%-65% range. However, we can see that when we add the condition “Age”, our accuracy increases to 70%. This shows that “Age” can help us filter out some non-compliant conditions in a very large testing dataset.

# 3. Computing the accuracy of prediction:
### 3.1Using K-NN algorithm

For the third part, it will be compute the accuracy of prediction by using K-NN algorithm.
And compare which one algorithm will be more accurate. Hence, before we apply the KNN algorithm, we have to understand
how can use the K-NN algorithm doing prediction.

We can Briefly review the K-NN algorithm: KNN is a nonparametric statistical method for classification and regression, which is often used with machine learning. So in our marathon project, it was used to determine whether a runner was male or female. We can also simply understand that the KNN algorithm tells us “look at what my neighbor says, and I'll go with it”. So we know the basic structure is that for a data point, the KNN will first focus on training our large database and select the “k” closest neighbors to itself, and then use the votes of these nearby neighbors to determine its own classification.

We can Briefly review the K-NN algorithm: KNN is a nonparametric statistical method for classification and regression, which is often used with maSo we will try  to use KNN algorithm to predict the gender. We do not have to modeling the 
probability distribution like in Bayes' theorem. Hence, let us understand how can we use the K_NN algorithm to make the prediction. 

* 1) Because we have big trian_set and test_set, we need to choose a number of neighbors.
* 2) Targeting a new data point, and measure the distance( utilize Euclidean) to the all training samples.
* 3) Find the k-closet training samples.
* 4) Ussing the majority vote to assign the prediction.chine learning. So in our marathon project, it was used to determine whether a runner was male or female. We can also simply understand that the KNN algorithm tells us “look at what my neighbor says, and I'll go with it”. So we know the basic structure is that for a data point, the KNN will first focus on training our large database and select the “k” closest neighbors to itself, and then use the votes of these nearby neighbors to determine its own classification.

### 3.12 For 1-Dimension "Time(s)"

In [320]:
# import the libraries again for convinence
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

features = ['Time(s)']

# Drop rows with missing data in 'Official Time' or 'Gender'
train_set_clean = train_set.dropna(subset=['M/F'] + features)
test_set_clean = test_set.dropna(subset=['M/F'] + features)

accuracies = []
for k in range(1, 16):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_set_clean[features], train_set_clean['M/F'])
    preds = knn.predict(test_set_clean[features])
    acc = accuracy_score(test_set_clean['M/F'], preds)
    accuracies.append((k, acc))

# Print results
for k, acc in accuracies:
    print(f"k = {k}, accuracy = {acc:.4f}")

k = 1, accuracy = 0.5736
k = 2, accuracy = 0.5787
k = 3, accuracy = 0.6008
k = 4, accuracy = 0.5966
k = 5, accuracy = 0.6133
k = 6, accuracy = 0.5952
k = 7, accuracy = 0.6093
k = 8, accuracy = 0.5994
k = 9, accuracy = 0.6203
k = 10, accuracy = 0.6103
k = 11, accuracy = 0.6190
k = 12, accuracy = 0.6091
k = 13, accuracy = 0.6205
k = 14, accuracy = 0.6141
k = 15, accuracy = 0.6221


### 3.13 For 2-dimension "Time(s)" and "Age"

In [299]:

# Define two features to use for KNN 
features = ['Time(s)', 'Age']

# Then we have to trainset the KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_set[features], train_set['M/F'])

# Predict using the test set
knn_predictions = knn.predict(test_set[features])

# we will predict the accuracy
knn_accuracy = accuracy_score(test_set['M/F'], knn_predictions)
print(f"KNN Classification Accuracy: {knn_accuracy:.4f}")


KNN Classification Accuracy: 0.7179


In [300]:
# Drop rows with missing female and male or missing values in the features
train_set_clean = train_set.dropna(subset=['M/F'] + features)
test_set_clean = test_set.dropna(subset=['M/F'] + features)


Then we will select the k nearest sample data by the steps III and IV described above and finally feel the classification based on the class distribution of the k sample data. Since our marathon database is too large, I have tried to select 1-15 sample distributions that can help us to better see the relevant accuracy data and the differences between them.

In [303]:

accuracies = []
for k in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_set_clean[features], train_set_clean['M/F'])
    preds = knn.predict(test_set_clean[features])
    acc = accuracy_score(test_set_clean['M/F'], preds)
    accuracies.append((k, acc))

# Display accuracies
for k, acc in accuracies:
    print(f"k = {k}, accuracy = {acc:.4f}")

k = 1, accuracy = 0.6774
k = 2, accuracy = 0.6595
k = 3, accuracy = 0.7076
k = 4, accuracy = 0.6971
k = 5, accuracy = 0.7179
k = 6, accuracy = 0.7125
k = 7, accuracy = 0.7253
k = 8, accuracy = 0.7264
k = 9, accuracy = 0.7287
k = 10, accuracy = 0.7251
k = 11, accuracy = 0.7359
k = 12, accuracy = 0.7333
k = 13, accuracy = 0.7382
k = 14, accuracy = 0.7354


Based on our data we can see that that accuracy rate is basically sort of sequential. Let's take an example: when we want to determine whether a running thing is a man or a woman. We already know that the age of the runner is 28 years old and the finishing time is 3 hours and 42 minutes. We can determine this by looking at the 5 runners whose ages and times match. The result is that 3 are male and 2 are female. Then the result will be judged as female.


# 4. NYC Marathon ( Bonus Point)

Our project will have extra part for the bonus points. We can get the most 
recent NYC marathon reocds at the webiste: https://www.nyrr.org/tcsnycmarathon/results/race-results. We can use the requests and Beautiful soup to scrape the top 10 finishers. 
According to the dataframe with the runners, we can through the different ways to analysis.
The elements could be: Name, Gender, Age, Place, Finishing Time.     

Then, for the bonus point part, we have to scrape top 10 finishers. Then we have to import the link
to help us figure out the data. First of all, inmport the libraries for our convinence.

In [310]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [311]:
# At first, we will use the url from the project 
url = 'https://www.nyrr.org/tcsnycmarathon/results/race-results'

In [312]:
# Then, we have to send HTTP request
HTML = requests.get(url)
# Then made the wonderful soup
Soup = BeautifulSoup(HTML.text, 'html.parser')


In [313]:
Soup # We show the soup(HTML from the Link)


<!DOCTYPE html>

<html class="no-js hide-debug" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="index, follow" name="robots"/>
<meta content="index, follow" name="googlebot"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<title>Race Results </title>
<link href="/favicon.ico" rel="shortcut icon"/>
<!-- icon in the highest resolution we need it for -->
<link href="/images/icons/icon.png" rel="icon" sizes="192x192"/>
<!-- reuse same icon for Safari -->
<link href="/images/icons/ios-icon.png" rel="apple-touch-icon"/>
<!-- multiple icons for IE -->
<meta content="/images/icons/icon_largetile.png" name="msapplication-square310x310logo"/>
<!-- Actual code for apple-touch -->
<link href="/images/icons/touch-icon-iphone.png" rel="apple-touch-icon"/>
<link href="https://fonts.goog

Then, when we have the HTML, for the nest step, we should inspect the structure of the table 
and try to extract the runners information. Like their name, gender, finish time, age and place.

In [316]:
runners = []
# Assume top 10 results are in a table format
table_rows = soup.select('table tr')[:11]  # header + 10 rows
for row in table_rows[1:]:
    cols = row.find_all('td')
    runner = {
        'Name': cols[1].text.strip(),
        'Gender': cols[2].text.strip(),
        'Age': int(cols[3].text.strip()),
        'Place': int(cols[0].text.strip()),
        'Finishing Time': cols[4].text.strip()
    }
    runners.append(runner)


In [317]:
# Convert to DataFrame
df_top10 = pd.DataFrame(runners)
print(df_top10)

Empty DataFrame
Columns: []
Index: []


# Conclusion 

This project demonstrates how can use Bayes' Theorem combined with KDE to constract a 1-dimenson classfier with element "Finish"
and the 2-dimension KDE with element "Finish" time and "Age" based on marathon. Also, this model performs very well, espically when we incorporating ages as the second feature.
And we can see by the resulting accuracy ratios that adding more features to a database with a large number of features allows for a gradual increase in prediction accuracy. Because this can help us to exclude some data that do not meet the conditions.

To assess our approach, we compared the Bayes classifier's performance to that of K-NN
classifier. When we using KNN, our results indicated that the number of neighbors k. 
KNN can show us more accurate prediction than the Bayes classifier, espically using multiple features.


For the furture exploration:
* We can try to add more features, for example : pace; 10K, 20K by using Bayes' theorem to observe the accuracy.
* Also we can choose to include uncertainties such as changes in the marathon environment, or weather and climate. We can use these uncertainties to look at the ratio of males to females in the Top10.